In [1]:
import urllib.request as req
from urllib.parse import urlparse, urlsplit
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import sqlite3
import re
pd.options.display.float_format = '{:,.1f}'.format

In [2]:
url = 'https://price.cetizen.com/'
res = req.Request(url)
html = req.urlopen(res).read()
soup = BeautifulSoup(html, 'html.parser')

In [3]:
wireless = {'wireless_1[]':'S', 'wireless_2[]':'K', 'wireless_3[]':'L', 'wireless_1,2[]':'S,K',
            'wireless_1,3[]':'S,L', 'wireless_2,3[]':'K,L', 'wireless_1,2,3[]':'S,K,L', 'wireless_7[]':'SELF'}

def getInfo(tag):
    tag2 = tag.find_all('li', {'style':re.compile('^float:left')})
    pno = urlparse(tag2[0].a['href']).query.split('&')[1].split('=')[1]
    name = tag2[0].text
    model = tag2[1].text
    price = tag2[2].text
    return pno, name, model, price

def getInfoWireless(wireless):
    rstList = []
    tag = soup.find_all('div', {'name':wireless[0]})
    for i in range(len(tag)):
        rstList.append([wireless[1], *getInfo(tag[i])])
    df = pd.DataFrame(rstList, columns={'wireless','pno','name','model','price'})
    return df

def getInfoAll():
    rstList = []
    for wl in wireless.items():        
        rstList.append(getInfoWireless(wl))
    df = pd.concat(rstList)
    return df

In [4]:
df = getInfoAll()

In [6]:
df

,price,pno,model,wireless,name
0,S,7366,LM-Y120S SKT,"138,000원",-
1,S,7314,SM-G981N SKT,"874,000원",-
2,S,7322,SM-G988N SKT,"1,071,000원",-
3,S,7318,SM-G986N SKT,"953,000원",-
4,S,7332,LM-Q510N SKT,"132,000원",-
...,...,...,...,...,...
51,SELF,7331,LM-Q510N 자급,"144,000원",-
52,SELF,7251,LM-V510N 자급,"556,000원","-48,000 ▼"
53,SELF,7134,LM-X625N 자급,"139,000원",-
54,SELF,7107,LM-X420N 자급,"73,000원",-
